In [1]:
# Transformer 다국어 기계 번역 모델 & 파인 튜닝 
# - Hugging Face 라이브러리 적용
# - AI HUB 금융 공시 정보 다국어 번역 데이터셋 적용
# - 입력된 문장을 다국어 기계 번역 모델을 통한 영어->한국어, 한국어->번역
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 금융 공시 정보 다국어 번역 데이터셋 전처리
# - 병렬 문장쌍 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [1]:
import torch
import tqdm as notebook_tqdm
import numpy as np
import glob, json, re, os, random, csv, zipfile

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

2.6.0 cpu
CUDA 사용 가능 여부: False
PyTorch CUDA 버전: None
빌드 정보: 2.6.0


In [3]:
# 데이터셋 전처리 - AI HUB 금융 공시 정보 다국어 번역 데이터셋
zip_path = './llm_data/TL_5. 공시정보_1. 영어.zip'
extract_dir = './llm_data/ai_hub_disclosure'

os.makedirs(extract_dir, exist_ok=True)

# 압축 풀기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print('압축 해제 완료:', extract_dir)

압축 해제 완료: ./llm_data/ai_hub_disclosure


In [4]:
# 데이터셋 전처리 - AI HUB 금융 공시 정보 다국어 번역 데이터셋
ko_lines, en_lines = [], []
folders = [ # 폴더 리스트 정의
    './llm_data/ai_hub_disclosure/*.json'
]

# 모든 JSON 읽기
for folder in folders:
    for path in glob.glob(folder): # 특정 디렉토리에서 지정한 패턴과 일치하는 모든 파일 경로를 리스트로 반환
        with open(path, encoding='utf-8') as f:
            try:
                data = json.load(f) # 파일 전체 로드(dict 구조)
            except json.JSONDecodeError:
                continue

            # sents 리스트에서 문장쌍 추출
            for sent in data.get('sents', []):
                en = sent.get('mtpe') # 원문(영어)
                ko = sent.get('source_cleaned') # 최종번역문(한국어) 추출

                if en and ko and ko != 'N/A':
                    en_lines.append(en.strip())
                    ko_lines.append(ko.strip())

# 1. Detokenize 함수 정의
def detokenize_sentence(sentence: str) -> str:
    sentence = sentence.strip()
    sentence = re.sub(r"\s+([?.!,])", r"\1", sentence)  # " ?" → "?"
    sentence = re.sub(r"\s+", " ", sentence)            # 여러 공백 → 하나
    return sentence

# 2. 데이터셋 전처리
en_lines = [detokenize_sentence(s) for s in en_lines]
ko_lines = [detokenize_sentence(s) for s in ko_lines]


print(f'총 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

총 문장쌍 개수: 80005, 80005
7회차 전환사채의 경우, 조정가액 적용일이 6월 26일이었으나 비영업일이었던 관계로 당일(6월 28일)을 적용일로 기재하였습니다.
For the 7th convertible bond, the adjustment date was June 26th. However, as it was a non-business day, the application date was recorded as the following day, June 28th.


In [5]:
# 데이터 전처리 - 중복 제거 및 순서 유지
# pairs = list(set(zip(en_lines, ko_lines)))
# en_lines, ko_lines = zip(*pairs) # 다시 분리
seen = set()
pairs = []
for en, ko in zip(en_lines, ko_lines):
    if (en, ko) not in seen:
        # 새로운 문장쌍을 집합에 기록, 이후 같은 문장쌍이 나오면 if 조건에서 걸러져 추가되지 않는다
        seen.add( (en, ko) )
        
        # 중복이 아닌 문장쌍을 리스트에 추가, 원래 순서대로 중복 없는 문장쌍 리스트가 만들어 진다
        # - pairs는 [("Hello","안녕"), ("Goodbye","잘가")] 
        pairs.append( (en, ko) )

# 이를 다시 분리 - 영어 문장들만 모아 ("Hello","Goodbye"), 한국어 문장들만 모아 ("안녕","잘가")
en_lines, ko_lines = zip(*pairs)

print(f'중복 제거 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

중복 제거 후 문장쌍 개수: 79991, 79991
7회차 전환사채의 경우, 조정가액 적용일이 6월 26일이었으나 비영업일이었던 관계로 당일(6월 28일)을 적용일로 기재하였습니다.
For the 7th convertible bond, the adjustment date was June 26th. However, as it was a non-business day, the application date was recorded as the following day, June 28th.


In [6]:
# 데이터 전처리 - 샘플링 추가

# 샘플링 최대 50,000 문장만 사용
# sample_size = 50000
sample_size = len(en_lines)
if len(ko_lines) > sample_size:
    indices = random.sample(range(len(ko_lines)), sample_size)
    ko_lines = [ ko_lines[i] for i in indices ]
    en_lines = [ en_lines[i] for i in indices ]

print(f'샘플링 후 문장쌍 개수: {len(ko_lines)}, {len(en_lines)}')
print(ko_lines[0])
print(en_lines[0])

샘플링 후 문장쌍 개수: 79991, 79991
7회차 전환사채의 경우, 조정가액 적용일이 6월 26일이었으나 비영업일이었던 관계로 당일(6월 28일)을 적용일로 기재하였습니다.
For the 7th convertible bond, the adjustment date was June 26th. However, as it was a non-business day, the application date was recorded as the following day, June 28th.


In [7]:
# 데이터 전처리 - 저장
out_dir = './llm_data/ai_hub_disclosure_translation'

# 폴더 없을시 생성
if not os.path.exists(out_dir):
    os.makedirs(out_dir, exist_ok=True)
    print(f'폴더 생성 완료: {out_dir}')
else:
    print(f'이미 존재하는 폴더: {out_dir}')

ko_path = f'{out_dir}/train_ko.txt'
en_path = f'{out_dir}/train_en.txt'

with open(ko_path, 'w', encoding='utf-8') as fko, \
    open(en_path, 'w', encoding='utf-8') as fen:
    for k, e in zip(ko_lines, en_lines):
        fko.write(k + '\n')
        fen.write(e + '\n')
print('저장 완료', ko_path, en_path)

폴더 생성 완료: ./llm_data/ai_hub_disclosure_translation
저장 완료 ./llm_data/ai_hub_disclosure_translation/train_ko.txt ./llm_data/ai_hub_disclosure_translation/train_en.txt


In [2]:
# AI Hub 금융 공시 정보 데이터셋(train_ko.txt, train_en.txt) -> CSV로 변환해 파인 튜닝
# 원본 데이터는 train_ko.txt, train_en.txt로 분리 -> 병렬 문장쌍을 만들어야 함
# 파인튜닝시 양방향 번역을 지원하려면 같은 문장쌍은 en->ko, ko->en 두방향으로 모두 포함해야 함
# CSV 구조 예시
# - src,tgt,src_lang,tgt_lang
# - You can buy it from a convenience store try it out.,편의점에서 사실 수 있으니 시도해보시길 바랍니다.,en,ko
# - 편의점에서 사실 수 있으니 시도해보시길 바랍니다.,You can buy it from a convenience store try it out.,ko,en
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model

# 전체 데이터 로드 - AI Hub 금융 공시 정보 데이터셋(train_ko.txt, train_en.txt)
with open('./llm_data/ai_hub_disclosure_translation/train_en.txt', 'r', encoding='utf-8') as f_en, \
    open('./llm_data/ai_hub_disclosure_translation/train_ko.txt', 'r', encoding='utf-8') as f_ko:
    en_lines = f_en.read().splitlines()
    ko_lines = f_ko.read().splitlines()

# 데이터 개수 제한 (예: 100개)
limit = 500
en_lines = en_lines[:limit]
ko_lines = ko_lines[:limit]

# train/valid split(90 : 10)
split_idx = int(len(en_lines) * 0.9)
train_en, valid_en = en_lines[:split_idx], en_lines[split_idx:]
train_ko, valid_ko = ko_lines[:split_idx], ko_lines[split_idx:]

print(len(train_en))
print(len(valid_en))

/Users/martin/AI/deep-learning/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/martin/AI/deep-learning/env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/martin/AI/deep-learning/env/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9B280146-BBD7-3F77-9873-F9740F2A5329> /Users/martin/AI/deep-learning/env/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <078DF3B7-D63C-3CEF-94C1-C063C5E5A468> /Users/martin/AI/deep-learning/env/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you hav

450
50


In [3]:
# 병렬 데이터 생성 - 편향되지 않은 데이터셋 생성
# 영어->한국어, 한국어->영어

# train.csv 생성
with open('./llm_data/ai_hub_disclosure_translation/train.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    print(train_en[0])
    print(train_ko[0])
    for en, ko in zip(train_en, train_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue

        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

# valid.csv 생성
with open('./llm_data/ai_hub_disclosure_translation/valid.csv', 'w', encoding='utf-8', newline='') as f_out:
    writer = csv.writer(f_out)
    writer.writerow(['src', 'tgt', 'src_lang', 'tgt_lang'])

    for en, ko in zip(valid_en, valid_ko):
        en, ko = en.strip(), ko.strip()
        if not en or not ko:
            continue
        
        # 영어 -> 한국어
        writer.writerow([en, ko, 'en', 'ko'])
        # 한국어 -> 영어
        writer.writerow([ko, en, 'ko', 'en'])

For the 7th convertible bond, the adjustment date was June 26th. However, as it was a non-business day, the application date was recorded as the following day, June 28th.
7회차 전환사채의 경우, 조정가액 적용일이 6월 26일이었으나 비영업일이었던 관계로 당일(6월 28일)을 적용일로 기재하였습니다.


In [4]:
# 토크나이저 전처리
from datasets import load_dataset # Hugging Face의 데이터셋 관리 라이브러리(학습용 데이터 로딩에 사용)
# M2M100 모델과 토크나이저, 학습 관련 유틸리티 제공
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model # LoRA 설정을 위한 라이브러리(모델 파라미터 효율적 파인튜닝)

# 1. tokenizer 로드, Hugging Face M2M100-418M 모델의 토크나이저 로드
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# 2. 전처리 함수 (동적 tgt_lang 설정)
def preprocess_function(examples):
    # 데이터셋에서 src(원문), tgt(번역문), tgt_lang(목표 언어 코드) 가져옴
    inputs = examples["src"]
    targets = examples["tgt"]
    tgt_langs = examples["tgt_lang"]

    model_inputs = tokenizer(
        inputs, # src(원문)을 토큰화
        max_length=128,  # 최대 길이 128
        truncation=True, # 최대 길이 초과시 잘라냄
        padding="max_length" # 길이가 부족하면 padding 처리
    )

    labels_list = []
    for text, lang in zip(targets, tgt_langs):
        # 각 문장마디 목표 언어(tgt_lang)
        if lang in tokenizer.lang_code_to_id:
            tokenizer.tgt_lang = lang
        else:
            tokenizer.tgt_lang = "en"  # 기본값
        
        # 타겟 문장 토큰화
        with tokenizer.as_target_tokenizer(): # 번역 대상 문장을 토큰화할때 사용하는 모드
            labels = tokenizer(
                text, # tgt(번역문)
                max_length=128, 
                truncation=True, 
                padding="max_length"
            )
        
        # labels["input_ids"]를 추출해서 학습용 정답(label)로 저장
        labels_list.append(labels["input_ids"])

    # 입력(src)와 정답(tgt)을 모두 포함한 딕셔너리 반환, Trainer가 이 반환값을 받아서 학습에 사용
    # {
    #     "input_ids": [...],        # 원문 토큰 시퀀스
    #     "attention_mask": [...],   # 패딩 여부 표시
    #     "labels": [...]            # 번역문 토큰 시퀀스
    # }
    model_inputs["labels"] = labels_list
    return model_inputs

In [5]:
# 데이터셋 불러오기
dataset = load_dataset( # Hugging Face datasets 라이브러리를 사용해 CSV 파일을 로드
    "csv",
    data_files={
        "train": "./llm_data/ai_hub_disclosure_translation/train.csv",
        "validation": "./llm_data/ai_hub_disclosure_translation/valid.csv"
    }
)

# 토크나이즈 적용
# - 결과: 각 샘플이 {"input_ids": ..., "attention_mask": ..., "labels": ...} 
tokenized_dataset = dataset.map( # map() 함수로 앞서 정의한 preprocess_function을 데이터셋 전체에 적용
    preprocess_function, 
    batched=True # 여러 샘플을 한번에 처리하여 속도 향상
)

Generating train split: 900 examples [00:00, 14405.99 examples/s]
Generating validation split: 100 examples [00:00, 13920.23 examples/s]
Map:   0%|          | 0/900 [00:00<?, ? examples/s]/Users/martin/AI/deep-learning/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 100/100 [00:00<00:00, 750.01 examples/s]


In [6]:
# 모델 로드

# pretrained model M2M100_418M 로드, 베이스 모델로 사용
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 설정
lora_config = LoraConfig(
    r=8,              # 랭크 크기(저차원 행렬 크기), 작을수록 가볍고 빠르지만 표현력이 줄어듬
    lora_alpha=32,    # 스케일링 계수, 학습된 LoRA 행렬을 원래 모델에 얼마나 반영할지 결정하는 스케일
    lora_dropout=0.1, # 드롭아웃
    target_modules=["q_proj", "v_proj"]  # Attention 모듈에 적용, Query/Value 저차원 행렬만 학습
)

# LoRA 모델 생성, PEFT(Param-Efficient Fine-Tuning) 모델 생성
model = get_peft_model(base_model, lora_config) # 전체 모델 파라미터는 그대로 두고 LoRA 모듈만 학습 대상이 됨

In [7]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="./llm_models/results_lora_ai_hub_disclosure",    # 학습 결과(모델, 체크포인트) 저장 경로
    eval_strategy="epoch",          # 구버전, 매 epoch마다 평가
    learning_rate=2e-4,             # 학습률 2e-5 - 5e-5
    per_device_train_batch_size=16, # 학습 배치 크기
    per_device_eval_batch_size=16,  # 평가 배치 크기
    num_train_epochs=3,             # 학습 epoch 수
    weight_decay=0.01,              # 가중치 감쇠(정규화)
    save_total_limit=2,             # 체크포인트 최대 저장 개수
    logging_dir="./llm_models/results_lora_logs_ai_hub_disclosure",           # 로그 저장 경로
    logging_steps=100,              # 100 step마다 로그 기록
)

# Trainer 정의
trainer = Trainer(
    model=model,        # LoRA 적용된 모델
    args=training_args, # 학습 설정
    train_dataset=tokenized_dataset["train"],       # 학습 데이터셋
    eval_dataset=tokenized_dataset["validation"],   # 평가 데이터셋
)

In [ ]:
# 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss


In [15]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 영어 → 한국어
text = "It is literally the basis of life."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))

# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
inputs = tokenizer(text, return_tensors="pt").to(device)   # 입력도 GPU로 이동
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id)
print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))

EN→KO: 그것은 문자 그대로 삶의 기초입니다.
KO→EN: It is literally the basis of life.


In [16]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/translation_model_ai_hub_disclosure")
tokenizer.save_pretrained("./llm_models/translation_model_ai_hub_disclosure")

('./llm_models/translation_model_ai_hub_disclosure/tokenizer_config.json',
 './llm_models/translation_model_ai_hub_disclosure/special_tokens_map.json',
 'llm_models/translation_model_ai_hub_disclosure/vocab.json',
 'llm_models/translation_model_ai_hub_disclosure/sentencepiece.bpe.model',
 './llm_models/translation_model_ai_hub_disclosure/added_tokens.json')

In [17]:
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
from peft import PeftModel

# 원본 M2M100 모델 로드
base_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

# LoRA 어댑터 붙여서 불러오기
model = PeftModel.from_pretrained(base_model, "./llm_models/translation_model_ai_hub_disclosure")

# 토크나이저도 불러오기
tokenizer = M2M100Tokenizer.from_pretrained("./llm_models/translation_model_ai_hub_disclosure")

# 디바이스 맞추기
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Glossary 분리
glossary_ko = {
    "이자율": "금리",
    "세금 자극": "세제 혜택",
    "부패 위험": "부도 위험",
    "자본 예비": "자본 준비금",
    "자본 보유량": "자본 준비금",
    "공급망 장애": "공급망 혼란",
    "산업의 풍경": "산업 구도",
    "노동-생명 경계": "일과 삶의 경계",
    "금리을": "금리를",
    "혼란를": "혼란을",
    "구도을": "구도를",
    "좁아졌다": "흐려졌다"
}

glossary_en = {
    "capital earnings": "capital reserves",
    "consumer trust": "consumer confidence",
    "inter-national": "international",
    "supply chain confusion": "supply chain disruptions",
    "unemployment risks": "default risks"
}

def postprocess_translation(text: str, glossary: dict) -> str:
    # 1. Glossary 치환
    for wrong, correct in glossary.items():
        text = text.replace(wrong, correct)

    # 2. 조사 중복 교정
    text = text.replace("을을", "을")
    text = text.replace("를를", "를")
    text = text.replace("을를", "를")
    text = text.replace("를을", "을")

    # 3. 금융 보고서 톤 교정
    # text = text.replace("떨어졌다", "하락했다")
    # text = text.replace("올랐다", "상승했다")
    # text = text.replace("줄였다", "축소했다")
    # text = text.replace("늘어났다", "확대됐다")
    # text = text.replace("~할 것입니다", "~할 것으로 예상된다")
    # text = text.replace("~될 것입니다", "~될 것으로 예상된다")

    # 4. 띄어쓰기/표현 보정
    # text = text.replace("했 습니다", "했습니다")
    # text = text.replace("했 다", "했다")
    # text = text.replace("것 입니다", "것입니다")
    # text = text.replace("재구성 할", "재구성할")

    return text



# 영어 → 한국어
text = "It is literally the basis of life."
texts_en = [
    "The company discloses its consolidated financial statements for Q3 2025.",
    "The board of directors has approved the resolution for entering a new business sector.",
    "The dividend payment plan for 2025 has been finalized at 500 KRW per share.",
    "According to the audit report, the internal accounting control system is operating appropriately.",
    "The largest shareholder’s ownership stake increased from 35% to 40%.",
    "As of December 31, 2025, the company reported total assets of 1 trillion KRW.",
    "The company decided to expand investment in its overseas subsidiaries.",
    "Sales revenue for the first half of 2025 increased by 15% compared to the same period last year.",
    "The company raised 50 billion KRW through the issuance of new bonds.",
    "The external auditor’s opinion was expressed as 'unqualified.'"
]
inputs = tokenizer(texts_en, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("EN→KO:")
for i, output in enumerate(outputs):
    decoded = tokenizer.decode(output, skip_special_tokens=True)
    processed = postprocess_translation(decoded, glossary_ko) # 후처리 적용
    print(f"{texts_en[i]} → {processed}")
    # print(f"{texts[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")

# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
texts_ko = [
    "회사는 2025년 3분기 연결재무제표를 공시합니다.",
    "당사는 신규 사업 진출을 위해 이사회 결의를 완료했습니다.",
    "2025년 배당금 지급 계획은 주당 500원으로 확정되었습니다.",
    "감사보고서에 따르면 내부회계관리제도는 적정하게 운영되고 있습니다.",
    "최대주주가 보유한 지분율은 35%에서 40%로 증가했습니다.",
    "회사는 2025년 12월 31일 기준 자산총액을 1조 원으로 보고했습니다.",
    "당사는 해외 자회사에 대한 투자 확대를 결정했습니다.",
    "2025년 상반기 매출액은 전년 동기 대비 15% 증가했습니다.",
    "회사는 신규 채권 발행을 통해 500억 원을 조달했습니다.",
    "외부감사인의 의견은 ‘적정’으로 표시되었습니다."
]
inputs = tokenizer(texts_ko, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\nKO→EN:")
for i, output in enumerate(outputs):
    decoded = tokenizer.decode(output, skip_special_tokens=True)
    processed = postprocess_translation(decoded, glossary_en) # 후처리 적용
    print(f"{texts_ko[i]} → {processed}")

    # print(f"{texts_ko[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


EN→KO:
The central bank decided to keep interest rates unchanged to stabilize market expectations. → 중앙은행은 시장 기대를 안정화하기 위해 금리를 변함없이 유지하기로 결정했다.
Major banks reported a decline in quarterly profits due to rising credit risks. → 주요 은행들은 신용 위험 증가로 인해 분기 수익이 줄어들었다고 보고했다.
The government introduced new fiscal policies to stimulate domestic consumption. → 정부는 국내 소비를 촉진하기 위해 새로운 세금 정책을 도입했습니다.
Stock markets rebounded after positive corporate earnings announcements. → 주식회사는 긍정적 인 기업 수익 발표 후 다시 묶여있다.
Financial regulators tightened disclosure requirements for listed companies. → 금융 규제 기관은 등록된 회사에 대한 공개 요구 사항을 강화했습니다.
Foreign investment surged following the signing of a new trade agreement. → 외국인 투자는 새로운 무역협정에 서명한 후에 상승했습니다.
The insurance industry anticipates growth amid increasing demand for health coverage. → 보험업계는 건강보험에 대한 수요가 증가함에 따라 성장할 것으로 예상된다.
The national currency depreciated against the dollar due to external market pressures. → 국가 통화는 외부 시장의 압박으로 인해 달러에 대한 가치가 낮아졌습니다.
Analysts forecast m

In [ ]:
# 베이스 모델 테스트
import torch

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 영어 → 한국어
text = "It is literally the basis of life."
texts = [
    "The company discloses its consolidated financial statements for Q3 2025.",
    "The board of directors has approved the resolution for entering a new business sector.",
    "The dividend payment plan for 2025 has been finalized at 500 KRW per share.",
    "According to the audit report, the internal accounting control system is operating appropriately.",
    "The largest shareholder’s ownership stake increased from 35% to 40%.",
    "As of December 31, 2025, the company reported total assets of 1 trillion KRW.",
    "The company decided to expand investment in its overseas subsidiaries.",
    "Sales revenue for the first half of 2025 increased by 15% compared to the same period last year.",
    "The company raised 50 billion KRW through the issuance of new bonds.",
    "The external auditor’s opinion was expressed as 'unqualified.'"
]
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["ko"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("EN→KO:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("EN→KO:")
for i, output in enumerate(outputs):
    print(f"{texts[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


# 한국어 → 영어
text = "말 그대로 삶의 기초입니다."
texts_ko = [
    "회사는 2025년 3분기 연결재무제표를 공시합니다.",
    "당사는 신규 사업 진출을 위해 이사회 결의를 완료했습니다.",
    "2025년 배당금 지급 계획은 주당 500원으로 확정되었습니다.",
    "감사보고서에 따르면 내부회계관리제도는 적정하게 운영되고 있습니다.",
    "최대주주가 보유한 지분율은 35%에서 40%로 증가했습니다.",
    "회사는 2025년 12월 31일 기준 자산총액을 1조 원으로 보고했습니다.",
    "당사는 해외 자회사에 대한 투자 확대를 결정했습니다.",
    "2025년 상반기 매출액은 전년 동기 대비 15% 증가했습니다.",
    "회사는 신규 채권 발행을 통해 500억 원을 조달했습니다.",
    "외부감사인의 의견은 ‘적정’으로 표시되었습니다."
]
inputs = tokenizer(texts_ko, return_tensors="pt", padding=True, truncation=True).to(device)
forced_bos_token_id = tokenizer.lang_code_to_id["en"]
# outputs = model.generate(**inputs, forced_bos_token_id=forced_bos_token_id, num_beams=5, max_length=128)
outputs = model.generate(
    **inputs,
    forced_bos_token_id=forced_bos_token_id,
    num_beams=5,
    max_length=256
)

# print("KO→EN:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("\nKO→EN:")
for i, output in enumerate(outputs):
    print(f"{texts_ko[i]} → {tokenizer.decode(output, skip_special_tokens=True)}")


EN→KO:
The central bank decided to keep interest rates unchanged to stabilize market expectations. → 중앙은행은 시장 기대를 안정화하기 위해 이자율을 변함없이 유지하기로 결정했다.
Major banks reported a decline in quarterly profits due to rising credit risks. → 주요 은행은 신용 위험 증가로 인해 분기 이익이 떨어지는 것으로 보고되었습니다.
The government introduced new fiscal policies to stimulate domestic consumption. → 정부는 국내 소비를 자극하기 위해 새로운 세금 정책을 도입했습니다.
Stock markets rebounded after positive corporate earnings announcements. → 주식 시장은 긍정적 인 기업 수익 발표 후 회복되었습니다.
Financial regulators tightened disclosure requirements for listed companies. → 금융 규제 기관은 등록된 회사에 대한 공개 요구 사항을 강화했습니다.
Foreign investment surged following the signing of a new trade agreement. → 외국인 투자는 새로운 무역 협정 서명 후 상승했다.
The insurance industry anticipates growth amid increasing demand for health coverage. → 보험 산업은 건강 보험에 대한 수요가 증가함에 따라 성장을 예상하고 있습니다.
The national currency depreciated against the dollar due to external market pressures. → 국가 통화는 외부 시장 압박으로 인해 달러에 비해 가치가 떨어졌습니다.
Analysts foreca

In [20]:
# sacrebleu.corpus_chrf는 chrF 점수를 계산하는 함수로, 번역된 문장(hypotheses)과 정답 문장(references) 간의 문자 단위 F-score를 측정
# BLEU 보다 문장 길이나 형태 변화에 덜 민감해서 번역 품질 평가에 자주 쓰인다
import sacrebleu

# LoRA 파인 튜닝 정수
# - EN→KO chrF Score:
# - chrF2 = 59.34
# - KO→EN chrF Score:
# - chrF2 = 69.97

# Base 모델 점수
# - EN→KO chrF Score:
# - chrF2 = 59.74
# - KO→EN chrF Score:
# - chrF2 = 70.77

# 영어 → 한국어 평가
references_ko = [ # 영어 문장의 정답 한국어 번역
    "중앙은행은 인플레이션을 억제하기 위해 금리를 인상하기로 결정했다.",
    "외국인 투자자들은 한국 국채에 강한 관심을 보이고 있다.",
    "회사는 수출 증가로 인해 분기별 수익이 크게 증가했다고 보고했다.",
    "신용 리스크 관리는 금융 안정성을 유지하는 데 필수적이다.",
    "새로운 규제 발표 이후 주식 시장은 높은 변동성을 경험했다.",
    "환율 변동은 다국적 기업에 큰 영향을 미친다.",
    "은행은 소매 고객을 위한 새로운 디지털 플랫폼을 도입했다.",
    "유동성 부족은 금융 부문에서 시스템 리스크로 이어질 수 있다.",
    "두 은행의 합병은 경쟁력을 향상시킬 것으로 예상된다.",
    "글로벌 금융 기관들은 미국의 통화 정책 영향을 면밀히 모니터링하고 있다."
]

hypotheses_ko = [ # 모델이 생성한 한국어 번역
    "중앙은행은 인플레이션을 통제하기 위해 이자율을 올리기로 결정했다.",
    "외국인 투자자들은 한국 정부의 부채에 강한 관심을 보이고 있습니다.",
    "회사는 더 높은 수출으로 인해 분기 수익이 상당히 증가했습니다.",
    "신용 위험 관리는 금융 안정성을 유지하는 데 필수적입니다.",
    "주식 시장은 새로운 규정 발표 후 높은 변동을 경험했습니다.",
    "교환율 변동은 다국적 기업에 큰 영향을 미칩니다.",
    "은행은 소매 고객을위한 새로운 디지털 플랫폼을 도입했습니다.",
    "유동성 결핍은 금융 부문에서 체계적인 위험을 초래할 수 있습니다.",
    "두 은행의 합병은 경쟁력을 향상시킬 것으로 예상된다.",
    "글로벌 금융 기관은 미국의 통화 정책의 영향을 철저히 모니터링하고 있습니다."
]

hypotheses_ko_lora= [
    "중앙은행은 인플레이션을 통제하기 위해 이자율을 올리기로 결정했다.",
    "외국인 투자자들은 한국 정부의 부채에 강한 관심을 보이고 있다.",
    "회사는 더 높은 수출으로 인해 분기 수익이 상당히 증가했습니다.",
    "신용 위험 관리는 금융 안정성을 유지하는 데 필수적입니다.",
    "주식 시장은 새로운 규정 발표 후 높은 변동을 경험했습니다.",
    "교환율 변동은 다국적 기업에 큰 영향을 미칩니다.",
    "은행은 소매 고객을위한 새로운 디지털 플랫폼을 도입했습니다.",
    "유동성 결핍은 금융 부문에서 체계적인 위험을 초래할 수 있습니다.",
    "두 은행의 합병은 경쟁력을 향상시킬 것으로 예상된다.",
    "세계 금융 기관은 미국의 통화 정책의 영향을 철저히 모니터링하고 있습니다."
]

print("EN→KO chrF Score:")
# chrF 점수를 계산
# chrf_ko = sacrebleu.corpus_chrf(hypotheses_ko, [references_ko])
chrf_ko = sacrebleu.corpus_chrf(hypotheses_ko_lora, [references_ko])
print(chrf_ko)


# 한국어 → 영어 평가
references_en = [
    "The central bank decided to raise interest rates to control inflation.",
    "Foreign investors are showing strong interest in Korean government bonds.",
    "The company reported a significant increase in quarterly earnings due to higher exports.",
    "Credit risk management is essential for maintaining financial stability.",
    "The stock market experienced high volatility after the announcement of new regulations.",
    "Exchange rate fluctuations have a major impact on multinational corporations.",
    "The bank introduced a new digital platform for retail customers.",
    "Liquidity shortages can lead to systemic risks in the financial sector.",
    "The merger between the two banks is expected to improve competitiveness.",
    "Global financial institutions are closely monitoring the impact of U.S. monetary policy."
]

hypotheses_en = [
    "The central bank has decided to raise interest rates to suppress inflation.",
    "Foreign investors are very interested in the Korean national debt.",
    "The company that a significant increase in quarterly revenue due to increased exports.",
    "Credit risk management is essential toining financial stability.",
    "Since the announcement of new regulations, the stock market has experienced high volatility.",
    "Currency rate variations have a great impact on multinational enterprises.",
    "The bank has introduced a new digital platform for retail customers.",
    "Lack of liquidity can lead to system risk in the financial sector.",
    "The fusion of the two banks is expected to boost competitiveness.",
    "Global financial institutions are closely monitoring the influence of U.S. currency policy."
]

hypotheses_en_lora= [
    "The central bank has decided to raise interest rates to suppress inflation.",
    "Foreign investors are very interested in the Korean national debt.",
    "The company that the quarterly revenue was significantly increased due to increased exports.",
    "Credit risk management is essential to maintain financial stability.",
    "After the announcement of new regulations, the stock market has experienced high volatility.",
    "The change in exchange rates has a significant impact on multinational companies.",
    "The bank has introduced a new digital platform for retail customers.",
    "Lack of liquidity can lead to system risk in the financial sector.",
    "The fusion of the two banks is expected to boost competitiveness.",
    "Global financial institutions are closely monitoring the influence of U.S. currency policy."
]

print("\nKO→EN chrF Score:")
# chrf_en = sacrebleu.corpus_chrf(hypotheses_en, [references_en])
chrf_en = sacrebleu.corpus_chrf(hypotheses_en_lora, [references_en])
print(chrf_en)

EN→KO chrF Score:
chrF2 = 59.34

KO→EN chrF Score:
chrF2 = 69.97
